<img src="https://www.inf.utfsm.cl/images/slides/Departamento-de-Informtica_HORIZONTAL.png" title="Title text" width="80%" />

<hr style="height:2px;border:none"/>

<H3 align='center'> MTI-PGE 2025 </H3>

<h1 align='center'>  Inteligencia Artificial y Aprendizaje Automático </h1>

<H3 align='center'> Trabajo Final </H3>

<H5 align='center'> Integrantes: </H3>

<H5 align='center'> Link al Video:  </H3>

<hr style="height:2px;border:none"/>





Link al desafío Kaggle https://www.kaggle.com/t/53be92513ca345a6a7454b1a91035358

## **Introducción**

Como en muchas otras áreas, la inteligencia artificial ha transformado significativamente el ámbito de la ciberseguridad, gracias a su capacidad para analizar grandes volúmenes de datos, modelar comportamientos complejos y detectar patrones asociados a actividades maliciosas. En particular, los enfoques basados en machine learning y deep learning han demostrado ser especialmente efectivos en la detección de anomalías en el tráfico de red, permitiendo identificar ataques sofisticados que pueden pasar inadvertidos para analistas humanos o sistemas tradicionales basados en reglas estáticas.

En esta tarea se proporciona el dataset UNSW-NB15, el cual contiene flujos de tráfico de red normal junto con múltiples categorías de ataques realistas y contemporáneos. Entre los escenarios de ataque presentes se incluyen, entre otros, DoS, Exploits, Reconnaissance, Fuzzers, Generic, Backdoors, Shellcode y Worms, representando una amplia variedad de técnicas utilizadas en entornos reales.

Cada flujo de red está descrito mediante un conjunto extenso de atributos que abarcan información básica de la comunicación, como direcciones IP y puertos de origen y destino, el protocolo de transporte utilizado (por ejemplo, TCP, UDP o ICMP), así como métricas temporales relacionadas con la duración del flujo. Adicionalmente, se incluyen características estadísticas y volumétricas, tales como el número de paquetes y bytes transmitidos, estados de conexión y otros indicadores de comportamiento relevantes para la detección de intrusiones.

El dataset permite abordar tanto clasificación binaria (tráfico normal vs. malicioso) como clasificación multiclase, diferenciando entre los distintos tipos de ataque. Su tarea consistirá en desarrollar un sistema basado en deep learning capaz de clasificar, de manera individual, los flujos de red presentes en el conjunto de pruebas (test), evaluando el desempeño del modelo en un escenario realista de detección de intrusiones. Para lo anterior usaremos el enfoque multiclase


## **Entregables**

* Se debe entregar el código utilizado en formato Jupyter notebook para poder **reproducir los resultados** presentados. El notebook debe estar seccionado y  ordenado para permitir identificar las celdas correspondientes a cada parte del trabajo y poder reproducir las predicciones subidas a Kaggle (si alguna pieza de la solución se carga pre-construida, deben incorporarse los archivos o los links necesarios para la ejecución completa del código).

* Se debe preparar un video de **10 a 15 minutos** donde se explique cómo se abordó el trabajo:

    - Se debe narrar brevemente la estrategia de solución adoptada enfocándose en aquello que los autores creen que la hará la solución ganadora.

    - Se descontarán 5 puntos por cada minuto excedido en el video.

    - En la cabecera del notebook pueden indicar un link al vide en youtube. Alternativamente pueden entregar un link a otro repositorio (e.g. Drive), pero por favor asegúrense de que podamos acceder al material. En caso de no poder acceder al video, la entrega se considerará no efectuada.

    - Se descontarán 5 puntos por cada día de atraso en la entrega.

* Se debe subir al menos una submission a la plataforma Kaggle como se ha hecho en clases. Su desempeño tendrá las siguientes bonificaciones o descuentos:

  - 1er Lugar = +10 pts.
  - 2do y 3er Lugar = +7 pts.
  - 4to Lugar = +5 pts.
  - 5to o más = +0 pts.

  - No hacer submission a Kaggle = -10 pts.
  - No superar el puntaje baseline = -5 pts.
    
* La solución debe incluir un mecanismo para evaluar el desempeño que tendrá la solución en datos nuevos, independientemente del puntaje que observe en Kaggle.

**IMPORTANTE: No olvide incluir al final de este notebook sus referencias al trabajo de otros que haya utilizado** El uso de sistemas como chatGPT, Claude, sCite u otros está permitido, pero debe indicarse cómo se usaron.

# Desarrollo
La estructura del trabajo está dividida en 4 bloques principales:

1.   Librerías
2.   Carga de los datos
3.   Modelo
4.   Validación

# Librerías



Se incorporan las librerías necesarias para ejecutar el proyecto

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from google.colab import files

import tensorflow as tf



# Carga de los datos

El data set se obtendrá desde Kaggle, para ello será necesario cargar el archivo de credenciales legacy de kaggle, este servirá para interactuar con la API y realizar la descarga.

Si no se dispone del archivo de credenciales, se puede generar uno accediendo a la configuración de la cuenta en https://www.kaggle.com/settings/account y presionando el botón "Create Legacy API Key".

In [2]:
#Cargamos el archivo de credenciales
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"leonardofuentes","key":"a74237f1488996f3cd47a44d39289c6c"}'}

In [3]:
# El CLI de Kaggle requiere de que el archivo de credenciales se encuentre en el
# directorio .kaggle del usuario que lo ejecuta, los siguientes comandos ubican
# el archivo en el lugar y con los permisos apropiados para su uso

# Crea directorio
os.makedirs('/root/.kaggle/', exist_ok=True)

# Mueve el archivo de credenciales
!mv kaggle.json /root/.kaggle/

# Estabece los permisos de lectura y escritura
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
# Probamos si la conexión a la API funciona revisando el leaderboard del desafío
# Este paso se puede omitir
!kaggle competitions leaderboard mti-2025-02-trabajo-final --show

  teamId  teamName               submissionDate              score    
--------  ---------------------  --------------------------  -------  
15062026  UNSW-MTI_baseline.csv  2025-12-23 19:18:46.510000  0.49631  
15059241  MTI KDM Active         2026-01-10 00:43:53.653000  0.19152  
15037259  Leonardo Fuentes       2026-01-11 02:26:11.240000  0.15511  


In [5]:
# Descargamos los archivos del desafío
!kaggle competitions download -c mti-2025-02-trabajo-final

# Dado que los archivos se encuentran en un empaquetado, es necesario extraerlos,
# los dejaremos en el directorio /content/data para futuros usos.
with zipfile.ZipFile('mti-2025-02-trabajo-final.zip', 'r') as zip_ref:
  zip_ref.extractall("/content/data")

  0% 0.00/140M [00:00<?, ?B/s]
100% 140M/140M [00:00<00:00, 1.52GB/s]


Una vez obtenidos los archivos con la data, entonces se procede a la carga de estos.

In [ ]:
# Definimos los tipos de datos de las columnas que pueden parecer ambiguos
tipos_especificos = {
    'sport': str,
    'dsport': str,
    'ct_ftp_cmd': str
}

# Cargamos los datos de entrenamiento desde el archivo UNSW-MTI_train.csv
dataTrain = pd.read_csv('/content/data/UNSW-MTI_train.csv', dtype=tipos_especificos)

# Cargamos los datos de prueba desde el archivo UNSW-MTI_test_set.csv
dataTest = pd.read_csv('/content/data/UNSW-MTI_test_set.csv')

In [ ]:
# Este paso se puede omitir

# Revisamos la carga realizada visualizando las primeras filas del dataset de
# entrenamiento y de test

print("\nPrimeras filas del dataset de entrenamiento: ",dataTrain.shape,"\n")
display(dataTrain.head())

print("\nPrimeras filas del dataset de prueba: ",dataTest.shape,"\n")
display(dataTest.head())


Primeras filas del dataset de entrenamiento:  (2100003, 49) 



,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,---,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,---,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,---,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,---,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,---,0



Primeras filas del dataset de prueba:  (440044, 48) 



,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ID
0,59.166.0.9,7045,149.171.126.7,25,tcp,FIN,0.201886,37552,3380,31,...,NaN,,2,2,7,4,1,1,3,0
1,59.166.0.9,9685,149.171.126.2,80,tcp,FIN,5.864748,19410,1087890,31,...,NaN,,3,1,4,4,1,1,1,1
2,59.166.0.2,1421,149.171.126.4,53,udp,CON,0.001391,146,178,31,...,NaN,,3,5,2,7,1,1,4,2
3,59.166.0.2,21553,149.171.126.2,25,tcp,FIN,0.053948,37812,3380,31,...,NaN,,1,1,4,7,1,1,3,3
4,59.166.0.8,45212,149.171.126.4,53,udp,CON,0.000953,146,178,31,...,NaN,,2,5,2,1,1,1,2,4


In [ ]:
print("\nCantidad de datos por categoría de ataque:")
print(dataTrain['attack_cat'].value_counts())


Cantidad de datos por categoría de ataque:
attack_cat
---                 1867614
Generic              153603
Exploits              33086
 Fuzzers              13805
DoS                   11446
 Reconnaissance        8698
 Fuzzers               5051
Analysis               2007
Reconnaissance         1759
Backdoor               1129
 Shellcode              917
Backdoors               534
Shellcode               223
Worms                   131
Name: count, dtype: int64


In [ ]:
# Identificamos que existen datos mal etiquetados, existiendo espacios antes y
# despues, así como tambien en singular y plural, por lo que procedemos a
# corregirlo

dataTrain['attack_cat'] = dataTrain['attack_cat'].str.strip()
dataTrain['attack_cat'] = dataTrain['attack_cat'].replace("Backdoor", "Backdoors")

# Con esto quedamos con las 10 categorías indicadas por la descripción del dataset
print(dataTrain['attack_cat'].value_counts())

attack_cat
---               1867614
Generic            153603
Exploits            33086
Fuzzers             18856
DoS                 11446
Reconnaissance      10457
Analysis             2007
Backdoors            1663
Shellcode            1140
Worms                 131
Name: count, dtype: int64


In [ ]:
# Guardamos los id del dataTest para usarlo en la submision a kaggle
test_ids = dataTest['ID'].copy()


In [ ]:
# Para los datasets, corregimos las columnas con problemas

# Columnas con problemas conocidos
cols_fix = ['ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd']

for df in [dataTrain, dataTest]:
    # Limpiar ct_ftp_cmd (strings vacíos o con espacios)
    df['ct_ftp_cmd'] = (
        df['ct_ftp_cmd']
        .astype(str)
        .str.strip()
        .replace('', np.nan)
    )

    # Imputación semánticamente correcta
    df['ct_flw_http_mthd'] = df['ct_flw_http_mthd'].fillna(0.0).astype(float)
    df['is_ftp_login'] = df['is_ftp_login'].fillna(0.0).astype(float)
    df['ct_ftp_cmd'] = df['ct_ftp_cmd'].fillna(0).astype(int)

In [ ]:
# Preparación de la variable objetivo
# Transformamos la columna attack_cat de texto a números, el encoder usará las
# categorías aprendidas con el dataset de entrenamiento en el de test.
le_attack_cat = LabelEncoder()
dataTrain['attack_cat_encoded'] = le_attack_cat.fit_transform(dataTrain['attack_cat'])

# Definimos la variable objetivo
y_train = dataTrain['attack_cat_encoded']

# Preparamos las variables de entrada, para ello eliminamos las columnas que no
# deben usarse como caracteristicas (Label, attack_cat y attack_cat_encoded) en
# el entrenamiento
X_train = dataTrain.drop(['Label', 'attack_cat', 'attack_cat_encoded'], axis=1)
X_test = dataTest.drop(['ID'], axis=1)

In [ ]:
# Revisamos las categorias de attack_cat
print(dict(enumerate(le_attack_cat.classes_)))

{0: '---', 1: 'Analysis', 2: 'Backdoors', 3: 'DoS', 4: 'Exploits', 5: 'Fuzzers', 6: 'Generic', 7: 'Reconnaissance', 8: 'Shellcode', 9: 'Worms'}


In [ ]:
# Tranformamos las variables de tipo objeto (como proto y service) en categorías
# numéricas
categorical_cols = X_train.select_dtypes(include='object').columns
category_maps = {}

for col in categorical_cols:
    unique_vals = X_train[col].astype(str).unique()
    category_maps[col] = {v: i for i, v in enumerate(unique_vals)}

for col in categorical_cols:
    mapping = category_maps[col]

    X_train[col] = X_train[col].astype(str).map(mapping)
    X_test[col] = X_test[col].astype(str).map(mapping)

    # Categorías no vistas → -1
    X_train[col] = X_train[col].fillna(-1).astype(int)
    X_test[col] = X_test[col].fillna(-1).astype(int)


# Modelo

Como vimos anteriormente, existe una gran diferencia entre la cantidad de datos normales v/s los que corresponden a ataque, esto amerita que la solución se enfoque en un modelo en cascada, donde en primera instancia se identifique si corresponde a un tráfico normal o ataque, para posteriormente analizar el tipo de ataque.

In [ ]:
# Modelo Binario

# Este modelo se encargará de distinguir entre tráfico normal o ataque.
# para ello se crean etiquetas binarias: 0 para normal, 1 para ataque, como la
# variable objetivo binaria.
y_train_bin = (dataTrain['attack_cat'] != '---').astype(int)

# Creamos un modelo binario
model_bin = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid') # Sigmoid para binario
])

#Entrenamos el modelo, asignadole mayor peso a la etiqueta de ataque.
model_bin.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_bin.fit(X_train, y_train_bin, epochs=10, batch_size=512, class_weight={0: 1, 1: 5})

Epoch 1/10
4102/4102 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.8433 - loss: 7650682.0000
Epoch 2/10
4102/4102 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8791 - loss: 403258.7500
Epoch 3/10
4102/4102 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8832 - loss: 371859.5312
Epoch 4/10
4102/4102 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8857 - loss: 333374.0625
Epoch 5/10
4102/4102 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8867 - loss: 313080.2188
Epoch 6/10
4102/4102 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8881 - loss: 289045.8125
Epoch 7/10
4102/4102 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8886 - loss: 267491.0625
Epoch 8/10
4102/4102 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8875 - loss: 269942.8438
Epoch 9/10
4102/4102 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8902 - loss: 231166.2344
Epoch 10/10
4102/4102 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8895 - loss: 225823.7812


In [ ]:
# Modelo especialista

# Este modelo identificará entre los tipos de ataque, para ello ajustamos las
# variables de entrada y objetivo excluyendo las filas que no correponden a ataques.
solo_ataques_mask = (dataTrain['attack_cat'] != '---')
X_train_ataques = X_train[solo_ataques_mask]
y_train_ataques = y_train[solo_ataques_mask] # Asegúrate de que y_train use el LabelEncoder

# Creamos el modelo especialista
model_especialista = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(le_attack_cat.classes_), activation='softmax')
])

# Dado que dentro de las clases de ataque aún existe mucha diferencia entre
# ellas (Generics v/s worm), definimos el peso para las etiquetas

from sklearn.utils import class_weight
# Calcular pesos: la clase minoritaria pesará mucho más
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_ataques),
    y=y_train_ataques
)
class_weights_dict = dict(enumerate(weights))

# Verás que el peso de la clase 0 (---) será muy bajo (~0.1)
# y el de Worms será muy alto (>1000)
print(class_weights_dict)

# Se entrena el modelo
model_especialista.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model_especialista.fit(X_train_ataques, y_train_ataques, epochs=20, batch_size=256, class_weight=class_weights_dict)

{0: np.float64(12.865470852017937), 1: np.float64(15.526758869512928), 2: np.float64(2.2558972566835576), 3: np.float64(0.7804207217554252), 4: np.float64(1.3693784471786168), 5: np.float64(0.16810218550418937), 6: np.float64(2.4692550444678205), 7: np.float64(22.65), 8: np.float64(197.10687022900763)}
Epoch 1/20
908/908 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 107567192.0000
Epoch 2/20
908/908 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 25360094.0000
Epoch 3/20
908/908 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 14346582.0000
Epoch 4/20
908/908 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 9081677.0000
Epoch 5/20
908/908 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 7226629.0000
Epoch 6/20
908/908 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 4748347.0000
Epoch 7/20
908/908 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2851602.7500
Epoch 8/20
908/908 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1012495.8750
Epoch 9/20
908/908 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 485.9779
Epoch 10/20
908/908 ━━━━━━━━━━━━━━━━━━━

In [ ]:
# 1. El modelo binario decide qué es sospechoso
prob_binaria = model_bin.predict(X_test)
es_ataque = (prob_binaria > 0.5).astype(int)

# 2. El modelo especialista predice el tipo de ataque para TODO el set
pred_especialista = model_especialista.predict(X_test).argmax(axis=1)

# 3. Lógica de Cascada:
# Si el modelo binario dice que es 0 (Normal), ponemos "---"
# Si dice que es 1 (Ataque), usamos la predicción del especialista
predicciones_finales_indices = []

for i in range(len(es_ataque)):
    if es_ataque[i] == 0:
        # Forzamos la etiqueta '---' (necesitas saber qué índice es en tu LabelEncoder)
        indice_normal = list(le_attack_cat.classes_).index('---')
        predicciones_finales_indices.append(indice_normal)
    else:
        predicciones_finales_indices.append(pred_especialista[i])

# 4. Convertir índices a nombres y guardar
final_names = le_attack_cat.inverse_transform(predicciones_finales_indices)

13752/13752 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step
13752/13752 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step


Evaluacion del modelo

In [ ]:

import pandas as pd
from keras.datasets import fashion_mnist
from google.colab import files

sample = pd.read_csv('/content/data/UNSW-MTI_sample_2.csv', dtype=str)
print(sample.columns.tolist())

df = sample.copy()

df['ID'] = test_ids.values.astype(str)
df['attack_cat'] = final_names.astype(str)

# Exportar a un archivo CSV
csv_filename = "predicciones.csv"
df.to_csv(
    csv_filename,
    index=False,
    encoding='utf-8',
    sep=','
)

# Descargar el archivo a la máquina local
#files.download(csv_filename)

!kaggle competitions submit -c mti-2025-02-trabajo-final -f predicciones.csv -m "Subido desde colab"

['ID', 'attack_cat']
100% 4.76M/4.76M [00:01<00:00, 2.57MB/s]
Successfully submitted to MTI - 2025-02 - Trabajo Final